In [44]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
import time
%matplotlib inline
from bs4 import BeautifulSoup as bs
import requests
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sqlalchemy import *
from sqlalchemy import create_engine
import calendar
import sqlite3 as sql
pd.options.display.float_format = '{:,.4f}'.format

init_notebook_mode(connected=True)

import os
main_dir = os.getcwd()

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')

from alphavantage import *
from yahoo_query import *
from option_slam_earnings import *
from reuters_query import reuters_query, reuters_insiders
from alphaquery import alphaquery

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Options')
from optionsFunctions import *

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Earnings')
from yahoo_earnings import *
from lookup_earnings import lookup_earnings

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Processing')
from hv_calc import *

os.chdir(main_dir)

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir(dbs_dir)

earn_engine = create_engine('sqlite:///earningsHistory.db', echo=False)
yahoo_engine = create_engine('sqlite:///yahoo.db', echo = False)
reuters_engine = create_engine('sqlite:///reuters.db', echo=False)

In [67]:
tickers = ['BBBY', 'LNN', 'AZZ', 'KBH', 'STZ', 'MSM', 'DAL', 'LEN']

annual_statements = {}
quarterly_statements = {}
profiles = []
query = 'SELECT * FROM {} WHERE Underlying IN ' + str(tuple(tickers))

for yahoo_table in yahoo_engine.table_names():
    if 'BalanceSheet' in yahoo_table:
        curr_table = pd.read_sql_query(query.format(yahoo_table), con = yahoo_engine,
                                       index_col = 'Underlying').drop_duplicates().fillna(np.nan)
        if 'annual' in yahoo_table:
            curr_table.year = pd.to_datetime(curr_table.year)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            annual_statements['balanceSheet'] = curr_table
        else:
            curr_table.quarter = pd.to_datetime(curr_table.quarter)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            quarterly_statements['balanceSheet'] = curr_table
            
    elif 'IncomeStatement' in yahoo_table:
        curr_table = pd.read_sql_query(query.format(yahoo_table), con = yahoo_engine,
                                       index_col = 'Underlying').drop_duplicates().fillna(np.nan)
        if 'annual' in yahoo_table:
            curr_table.year = pd.to_datetime(curr_table.year)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            annual_statements['incomeStatements'] = curr_table
        else:
            curr_table.quarter = pd.to_datetime(curr_table.quarter)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            quarterly_statements['incomeStatements'] = curr_table
            
    elif 'CashFlow' in yahoo_table:
        curr_table = pd.read_sql_query(query.format(yahoo_table), con = yahoo_engine,
                                       index_col = 'Underlying').drop_duplicates().fillna(np.nan)
        if 'annual' in yahoo_table:
            curr_table.year = pd.to_datetime(curr_table.year)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            annual_statements['cashFlows'] = curr_table
        else:
            curr_table.quarter = pd.to_datetime(curr_table.quarter)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            quarterly_statements['cashFlows'] = curr_table
    elif 'profiles' in yahoo_table:
        curr_table = pd.read_sql_query(query.format(yahoo_table), con = yahoo_engine,
                                       index_col = 'Underlying').drop_duplicates()[['industry', 'sector']]
        profiles.append(curr_table)
    else:
        continue
profiles = pd.concat(profiles, axis = 0)

In [70]:
for key,val in annual_statements.items():
    print(key)
    print(val.columns)

balanceSheet
Index(['year', 'accountsPayable', 'capitalSurplus', 'cash', 'commonStock',
       'deferredLongTermAssetCharges', 'deferredLongTermLiab', 'goodWill',
       'intangibleAssets', 'inventory', 'longTermDebt', 'longTermInvestments',
       'maxAge', 'minorityInterest', 'netReceivables', 'netTangibleAssets',
       'otherAssets', 'otherCurrentAssets', 'otherCurrentLiab', 'otherLiab',
       'otherStockholderEquity', 'propertyPlantEquipment', 'retainedEarnings',
       'shortLongTermDebt', 'shortTermInvestments', 'totalAssets',
       'totalCurrentAssets', 'totalCurrentLiabilities', 'totalLiab',
       'totalStockholderEquity', 'treasuryStock'],
      dtype='object')
cashFlows
Index(['year', 'capitalExpenditures', 'changeInCash',
       'changeToAccountReceivables', 'changeToInventory',
       'changeToLiabilities', 'changeToNetincome',
       'changeToOperatingActivities', 'depreciation', 'dividendsPaid',
       'effectOfExchangeRate', 'investments', 'issuanceOfStock',
       '